## Optimizers

### Gradient descent

The Core Problem: Gradient Descent is memoryless. As it gets closer to the minimum and the gradient shrinks, its steps become smaller and smaller, causing it to slow down dramatically.

Let's use the simple function f(p) = p**2. The minimum is at p=0, and the gradient is f'(p) = 2p. We start at p=10 and use a learning rate of η = 0.1.

In [1]:
p=10
lr=0.1
gradient=2*p
f=p**2


for  i in range(10):
    p_prev=p
    gradient_n=2*p
    update = lr*gradient_n
    p=p-update
    print(f"iter: {i} cur p: {p_prev:.2f} gradient: {gradient_n:.4f} update(lr*g): {update:.4f} new p {p:.2f}  ")


iter: 0 cur p: 10.00 gradient: 20.0000 update(lr*g): 2.0000 new p 8.00  
iter: 1 cur p: 8.00 gradient: 16.0000 update(lr*g): 1.6000 new p 6.40  
iter: 2 cur p: 6.40 gradient: 12.8000 update(lr*g): 1.2800 new p 5.12  
iter: 3 cur p: 5.12 gradient: 10.2400 update(lr*g): 1.0240 new p 4.10  
iter: 4 cur p: 4.10 gradient: 8.1920 update(lr*g): 0.8192 new p 3.28  
iter: 5 cur p: 3.28 gradient: 6.5536 update(lr*g): 0.6554 new p 2.62  
iter: 6 cur p: 2.62 gradient: 5.2429 update(lr*g): 0.5243 new p 2.10  
iter: 7 cur p: 2.10 gradient: 4.1943 update(lr*g): 0.4194 new p 1.68  
iter: 8 cur p: 1.68 gradient: 3.3554 update(lr*g): 0.3355 new p 1.34  
iter: 9 cur p: 1.34 gradient: 2.6844 update(lr*g): 0.2684 new p 1.07  


### Gradient Descent with Momentum

In [ ]:
p=10
lr=0.1
gradient=2*p
f=p**2
velocity=0
beta=0.5


for  i in range(10):
    gradient_n=2*p
    velocity = 0.5 * velocity + gradient_n
    update = lr*velocity
    p=p-update
    print(f"iter: {i} cur p: {p:.2f} gradient: {gradient_n:.4f} velocity: {velocity:.2f} update(lr*g): {update:.4f} new p {p:.2f}  ")

### Inflexible learning rates

The Core Problem: Momentum helps find a better direction, but it's still handicapped by a single, global learning rate. This fails when parameters have vastly different sensitivities.

f(p) = 50 * p[0]**2 + p[1]**2

The minimum is at (0, 0). The gradient vector is:

∂f/∂p[0] = 100 * p[0]

∂f/∂p[1] = 2 * p[1]

The gradient for p[0] is 50 times stronger than for p[1]. This means p[0] is an extremely "sensitive" parameter, while p[1] is "stubborn."

To prevent the first p (p[0]) from never converging, a small learning rate is needed, but this makes the convergence of p[1] very slow.

In [ ]:
p=[1.5,10]
lr=0.01
gradient_0=2*p
gradient_1=100*p
#f=50*(p0**2)+(p1*2)


for  i in range(10):
    p_prev=p.copy()
    gradient_0_n=100*p[0]
    gradient_1_n=2*p[1]
    update_0 = lr*gradient_0_n
    update_1 = lr*gradient_1_n
    p[0]=p[0]-update_0
    p[1]=p[1]-update_1
    print(f"iter: {i} cur p: {p_prev[0]:.2f},{p_prev[1]:.2f} gradient: {gradient_n:.4f} update(lr*g): {update_0:.4f}, {update_1:.4f} new p {p[0]:.2f} {p[1]:.2f}")

### Adaptive learning rates for every parameter

AdaGrad gives each parameter its own learning rate that adapts over time. It does this by dividing the base learning rate by the square root of the sum of all past squared gradients for that parameter. But  as the sum of all past squared gradients keeps growing, the adapted learning rate eventually will diminish to zero.

In [ ]:
import math
g_squared=[0,0]
eps = 0.0000001

p=[1.5,10]
lr=1.5
gradient_0=2*p
gradient_1=100*p
#f=50*(p0**2)+(p1*2)


for  i in range(10):
    p_prev=p.copy()
    gradient_0_n=100*p[0]
    gradient_1_n=2*p[1]
    g_squared[0]+=gradient_0_n**2
    g_squared[1]+=gradient_1_n**2 
    adapted_lr_0 = lr/(math.sqrt(g_squared[0]) + eps)
    adapted_lr_1 = lr/(math.sqrt(g_squared[1]) + eps)
    update_0 = adapted_lr_0*gradient_0_n
    update_1 = adapted_lr_1*gradient_1_n
    p[0]=p[0]-update_0
    p[1]=p[1]-update_1
    print(f"iter: {i} cur p: {p_prev[0]:.2f},{p_prev[1]:.2f} gradient: {gradient_0_n:.4f},{gradient_1_n:.4f} g_squared {g_squared[0]:.2f}, {g_squared[1]:.2f}  adapted lr:{adapted_lr_0:.4f}, {adapted_lr_1:.4f} update(lr*g): {update_0:.4f}, {update_1:.4f} new p {p[0]:.2f} {p[1]:.2f}")

### Adam

Adam is an optimizer that combines solutions like momentum and adaptive learning rates in one optimizer. Without introducing the 'dyying' learning rate anew. Adam achieves this by using a more flexible memory system for both direction and magnitude.

>Momentum can be described as:
The direction is remembered for the last few steps, even the function hits a small plateau, the optimizer will remember the  direction from when the function had a steeper decay.

>Adaptive learning rates can be described as:
When functions develop steeper, small, cautious steps (divide by a large number) are used. If at some point a function is flat, larger learning steps are taken to move faster (divide by a small number).

>Bias correction
Because learning starts with no history, it "corrects" itself during the first few learning steps so it doesn't stay too conservative or get stuck near the starting point.

"Adam is like a GPS that not only knows where to go but also remembers your past speed, adapts to the current terrain, and corrects itself to take the fastest, safest route to the bottom."


The optimizer's calculations will look like 

**Gradients:**

$$ g_{t}=\nabla _{\theta }f_{t}(\theta _{t-1})  $$

**First moment (moving average of the gradients)**

$$ m_{t}=\beta _{1}\cdot m_{t-1}+(1-\beta _{1})\cdot g_{t} $$

**Second moment (moving average of the squared gradients):**

$$ v_{t}=\beta _{2}\cdot v_{t-1}+(1-\beta _{2})\cdot g_{t}^{2} $$

**Bias corrected moments:**

$$ \^{m}_{t}=\frac{m_{t}}{1-\beta _{1}^{t}} $$

$$ \^{v}_{t}=\frac{v_{t}}{1-\beta _{2}^{t}} $$

**Updating the parameters:**

$$ \theta _{t}=\theta _{t-1}-\frac{\alpha \cdot \^{m}_{t}}{\sqrt{\^{v}_{t}}+\epsilon } $$


**Parameters:**\
$\alpha$: Learning rate, typically 0.001.\
$\beta _{1}$: Decay rate for the first moment, typically \(0.9\).\
$\beta _{2}$: Decay rate for the second moment, typically \(0.999\).\
$\epsilon$: A small constant (eg. \(10^{-8}\)) to prevent division by zero.\
$t$: the current time step\
$t-1$: the previous time step\
$g$:  the gradient


### Gradient descent fails

A situation where gradient descent fails for the simple one p**2 function. When choosing a learning rate of higher than 1.05, SGD fails to find the minimum.

In [9]:
p=10
lr=1.05
gradient=2*p
f=p**2


for  i in range(10):
    p_prev=p
    gradient_n=2*p
    update = lr*gradient_n
    p=p-update
    print(f"iter: {i} cur p: {p_prev:.2f} gradient: {gradient_n:.4f} update(lr*g): {update:.4f} new p {p:.2f}  ")

iter: 0 cur p: 10.00 gradient: 20.0000 update(lr*g): 21.0000 new p -11.00  
iter: 1 cur p: -11.00 gradient: -22.0000 update(lr*g): -23.1000 new p 12.10  
iter: 2 cur p: 12.10 gradient: 24.2000 update(lr*g): 25.4100 new p -13.31  
iter: 3 cur p: -13.31 gradient: -26.6200 update(lr*g): -27.9510 new p 14.64  
iter: 4 cur p: 14.64 gradient: 29.2820 update(lr*g): 30.7461 new p -16.11  
iter: 5 cur p: -16.11 gradient: -32.2102 update(lr*g): -33.8207 new p 17.72  
iter: 6 cur p: 17.72 gradient: 35.4312 update(lr*g): 37.2028 new p -19.49  
iter: 7 cur p: -19.49 gradient: -38.9743 update(lr*g): -40.9231 new p 21.44  
iter: 8 cur p: 21.44 gradient: 42.8718 update(lr*g): 45.0154 new p -23.58  
iter: 9 cur p: -23.58 gradient: -47.1590 update(lr*g): -49.5169 new p 25.94  


### Adam

When using Adam in the same situation, Adam is capable of handling the very high learning rate 

In [31]:
import math
p=10
lr=1.05
gradient=2*p
f=p**2
m = 0
v = 0
beta1=0.9
beta2=0.999
eps=0.00000001


for  i in range(10):
    p_prev=p
    gradient_n=2*p
    m = (beta1*m) + (1-beta1)*gradient_n
    m_hat = m / (1-(beta1**(i+1)))
    v = (beta2*v) + (1-beta2)*(gradient_n**2)
    v_hat= v/ (1-(beta2**(i+1)))
    update = lr*(m_hat/(math.sqrt(v_hat)+eps))
    p=p-update
    print(f"iter: {i} cur p: {p_prev:.2f} gradient: {gradient_n:.4f} moment 1: {m:.2f} moment 2: {v:.2f} m_hat {m_hat:.2f} v_hat: {v_hat:.2f} update(lr*g): {update:.4f} new p {p:.2f}  ")

iter: 0 cur p: 10.00 gradient: 20.0000 moment 1: 2.00 moment 2: 0.40 m_hat 20.00 v_hat: 400.00 update(lr*g): 1.0500 new p 8.95  
iter: 1 cur p: 8.95 gradient: 17.9000 moment 1: 3.59 moment 2: 0.72 m_hat 18.89 v_hat: 360.19 update(lr*g): 1.0454 new p 7.90  
iter: 2 cur p: 7.90 gradient: 15.8093 moment 1: 4.81 moment 2: 0.97 m_hat 17.76 v_hat: 323.40 update(lr*g): 1.0367 new p 6.87  
iter: 3 cur p: 6.87 gradient: 13.7358 moment 1: 5.70 moment 2: 1.16 m_hat 16.59 v_hat: 289.67 update(lr*g): 1.0233 new p 5.84  
iter: 4 cur p: 5.84 gradient: 11.6891 moment 1: 6.30 moment 2: 1.29 m_hat 15.39 v_hat: 259.00 update(lr*g): 1.0042 new p 4.84  
iter: 5 cur p: 4.84 gradient: 9.6808 moment 1: 6.64 moment 2: 1.38 m_hat 14.17 v_hat: 231.38 update(lr*g): 0.9783 new p 3.86  
iter: 6 cur p: 3.86 gradient: 7.7242 moment 1: 6.75 moment 2: 1.44 m_hat 12.94 v_hat: 206.78 update(lr*g): 0.9446 new p 2.92  
iter: 7 cur p: 2.92 gradient: 5.8350 moment 1: 6.66 moment 2: 1.48 m_hat 11.69 v_hat: 185.11 update(lr*g)